In [107]:
from bs4 import BeautifulSoup

# https://app.roll20.net/campaigns/chatarchive/8981579?p=1&onePage=true&hidewhispers=&hiderollresults=
# save page

# (pagenate https://app.roll20.net/campaigns/chatarchive/8981579?p=1 with cache?)
# annoying data seems to come from graphql responses
with open('data/ChatLog_SecondForm_1_4_2022.html', mode='r', encoding="utf8") as fd:
    html_doc = fd.read()

soup = BeautifulSoup(html_doc, 'html.parser')

In [108]:
messages = soup.find_all('div', 'message')

print(len(messages))

1536


In [109]:
def is_generic_roll(message_class_list):
    return True if 'rollresult' in message_class_list else False
    
def is_new_character_rolling(message):
    return message.find(class_='spacer')

In [110]:
from dateutil.parser import parse

current_time = None
timestamps = []
for message in messages:
    tstamp = message.find(class_='tstamp')
    if tstamp:
        current_time = parse(tstamp.string)
    timestamps.append(current_time)

print(len(timestamps))
print(timestamps[-20:])

1536
[datetime.datetime(2022, 1, 4, 0, 17), datetime.datetime(2022, 1, 4, 0, 17), datetime.datetime(2022, 1, 4, 0, 17), datetime.datetime(2022, 1, 4, 0, 17), datetime.datetime(2022, 1, 4, 0, 17), datetime.datetime(2022, 1, 4, 0, 17), datetime.datetime(2022, 1, 4, 0, 17), datetime.datetime(2022, 1, 4, 0, 17), datetime.datetime(2022, 1, 4, 0, 17), datetime.datetime(2022, 1, 4, 0, 17), datetime.datetime(2022, 1, 4, 0, 17), datetime.datetime(2022, 1, 4, 0, 17), datetime.datetime(2022, 1, 4, 0, 17), datetime.datetime(2022, 1, 4, 0, 17), datetime.datetime(2022, 1, 4, 0, 17), datetime.datetime(2022, 1, 4, 0, 17), datetime.datetime(2022, 1, 4, 0, 17), datetime.datetime(2022, 1, 4, 0, 17), datetime.datetime(2022, 1, 4, 0, 17), datetime.datetime(2022, 1, 4, 0, 17)]


In [111]:
has_roll = []
for message in messages:
    has_roll.append((message.find(class_='inlinerollresult') is not None) or ('rollresult' in message['class']))

print(len([i for i in has_roll if i ]))

# roll_timestamps = [timestamp for i, timestamp in zip(has_roll, timestamps) if i]
roll_msgs = [message for i, message, in zip(has_roll, messages) if i ]

965


In [112]:
import re
characters = [] # Who rolled
roll_types = [] # What kind of roll: (1) Ability Check (2) Saving Throw (3) Initiative (4) Spell/Attack
d20_rolls = [] # Filter rolls made by 1d20 rolls only

roll_re = re.compile(r'">([0-9]+)</span>')

name_dictionary = {
    'Kal': 'Kal',
    'Eva Y.': 'Kal',
    'Kalju Dormaris': 'Kal',
    '(To GM)': 'Kal',
    'Feyrn': 'Feyrn',
    'Larissa J.': 'Feyrn',
    'Katie H. (GM)': 'NPC',
    'Alejandra E.': 'Leslie',
    'Leslie Timbles': 'Leslie',
    'Val C.': 'Viktor',
    'Viktor': 'Viktor',
    'Dionne Z.': 'Damaia',
    'Damaia': 'Damaia',
    'Leslie/Viktor': 'Leslie'
}

playerid_dictionary = {
    'player--LYJ1cGjM-W040RlsKb2': 'Feyrn',
    'player--LYJ5dm7p5gBuuB19Pl5': 'Leslie',
    'player--LYJAxZP302qwjkWB3XC': 'Damaia',
    'player--LYJ0fhiIzk0Cr-lw1nm': 'Viktor',
    'player--MCzBHRIPXyWAqzxOYqb': 'Kal',
    'player--LYJ-xnOUU_pobjJKmN4': 'NPC'
}

roll_dictionary = {
    'ATHLETICS' : 'Ability Check',
    'ACROBATICS' : 'Ability Check',
    'SLEIGHT OF HAND' : 'Ability Check',
    'STEALTH' : 'Ability Check',
    'ARCANA' : 'Ability Check',
    'HISTORY' : 'Ability Check',
    'INSIGHT' : 'Ability Check',
    'PERFORMANCE' : 'Ability Check',
    'DECEPTION' : 'Ability Check',
    'PERSUASION' : 'Ability Check',
    'ANIMAL HANDLING' : 'Ability Check',
    'MEDICINE' : 'Ability Check',
    'SURVIVAL' : 'Ability Check',
    'PERCEPTION' : 'Ability Check',
    'INTIMATION' : 'Ability Check',
    'RELIGION' : 'Ability Check',
    'INVESTIGATION' : 'Ability Check',
    'NATURE' : 'Ability Check',
    'STRENGTH' : 'Ability Check',
    'DEXTERITY' : 'Ability Check',
    'CONSTITUTION' : 'Ability Check',
    'INTELLIGENCE' : 'Ability Check',
    'WISDOM' : 'Ability Check',
    'CHARISMA' : 'Ability Check',
    'INITIATIVE' : 'Initiative'  
}

roll_timestamps = []

current_char = ''
    
for message in roll_msgs:    
    is_generic = is_generic_roll(message['class'])
    
    if is_generic:
        # Who
        character_id = ''
        
        for _class in message['class']:
            if 'player--' in _class:
                character_id = _class
        
        character = playerid_dictionary[character_id] # Human-readable conversion
        current_char = character # Update

        # Filter 1d20 only; Amount
        if 'rolling d20' in message.find(class_='formula').get_text():
            amnt = int(message.find(class_='rolled').get_text())
            
            # Append
            characters.append(character)
            roll_types.append('Generic')
            d20_rolls.append(amnt)
            tstamp = message.find(class_='tstamp')
            if tstamp:
                current_time = parse(tstamp.string)
            roll_timestamps.append(current_time)
        
    else:
        character = ''
        
        # Who
        if message.find(class_='you'):
            character = 'Kal'
        elif message.find(class_='spacer'):
            current_char = message.find(class_='by').get_text().strip()[:-1]
            character = name_dictionary[current_char]
        else:
            character = name_dictionary[current_char]
            
        # What type; Amount
        inline_span = message.find('span', class_='inlinerollresult')
        if 'Rolling 1d20' in inline_span['title']:
            roll_txt = inline_span['title']
            amnt = int(roll_re.search(roll_txt).group(1))
            
            sheet_label = message.find(class_='sheet-label')
            roll_type_txt = sheet_label.find('span').get_text().split('(')[0].strip().upper()
            roll_type = ''
            
            if roll_type_txt in roll_dictionary:
                roll_type = roll_dictionary[roll_type_txt]
            elif 'SAVE' in roll_type_txt:
                roll_type = 'Saving Throw'
            else:
                roll_type = 'Spell/Attack'
            
            characters.append(character)
            roll_types.append(roll_type)
            d20_rolls.append(amnt)
            tstamp = message.find(class_='tstamp')
            if tstamp:
                current_time = parse(tstamp.string)
            roll_timestamps.append(current_time)
            

print('Characters:', len(characters), characters)
print('Roll Types:', len(roll_types), roll_types)
print('d20 rolls:', len(d20_rolls), d20_rolls)
print('Timestamps:', len(roll_timestamps), roll_timestamps)

Characters: 798 ['Leslie', 'Leslie', 'Viktor', 'Leslie', 'Viktor', 'Feyrn', 'Viktor', 'Viktor', 'Feyrn', 'Viktor', 'Feyrn', 'Feyrn', 'Damaia', 'Leslie', 'Feyrn', 'Viktor', 'Damaia', 'Viktor', 'Feyrn', 'Damaia', 'Viktor', 'Feyrn', 'Viktor', 'Damaia', 'Feyrn', 'Viktor', 'Feyrn', 'Leslie', 'Damaia', 'Viktor', 'Viktor', 'Damaia', 'Feyrn', 'Feyrn', 'Leslie', 'Damaia', 'Leslie', 'Viktor', 'Damaia', 'Feyrn', 'Damaia', 'Viktor', 'Leslie', 'Feyrn', 'Feyrn', 'Damaia', 'Viktor', 'Feyrn', 'Leslie', 'Viktor', 'Feyrn', 'Damaia', 'Damaia', 'Leslie', 'Viktor', 'Feyrn', 'Damaia', 'Viktor', 'Feyrn', 'Damaia', 'Leslie', 'Viktor', 'Feyrn', 'Leslie', 'Viktor', 'Damaia', 'Feyrn', 'Leslie', 'Feyrn', 'Viktor', 'Leslie', 'Viktor', 'Damaia', 'Feyrn', 'Feyrn', 'Leslie', 'Viktor', 'Viktor', 'Leslie', 'Damaia', 'Feyrn', 'Feyrn', 'Feyrn', 'Feyrn', 'Feyrn', 'Feyrn', 'Feyrn', 'Feyrn', 'Feyrn', 'Feyrn', 'Feyrn', 'Feyrn', 'Viktor', 'Leslie', 'Viktor', 'Damaia', 'Leslie', 'Viktor', 'Viktor', 'Leslie', 'Viktor', 'Damaia'

In [113]:
from datetime import timedelta, datetime

MIN_ROLLS = 20
MAX_GAP = timedelta(hours=12)

start_time = roll_timestamps[0]
last_time = roll_timestamps[0]
count = 1

sessions = []

session_idx = []

for time in roll_timestamps:
    session_idx.append(len(sessions))
    if time - last_time > MAX_GAP:
        if count > MIN_ROLLS:
            sessions.append((start_time, last_time, count))
            print(last_time-start_time, count)
        count = 1
        start_time = time
    else:
        count += 1
    last_time = time
if count > MIN_ROLLS:
    sessions.append((start_time, last_time, count))
    print(last_time-start_time, count)

with open('data/sessions.txt', 'w') as fd:
    for session in sessions:
        fd.write(session[0].strftime('%Y-%m-%d\n'))

5:47:00 124
4:15:00 125
3:54:00 25
3:54:00 86
3:28:00 35
4:34:00 60
4:46:00 78
2:28:00 28
3:28:00 39
3:47:00 39
2:41:00 26
4:45:00 38
1:58:00 57


In [114]:
import pandas as pd

roll_df = pd.DataFrame({
    'time': roll_timestamps,
    'character': characters,
    'session': session_idx,
    'type': roll_types,
    'value': d20_rolls
})

roll_df = roll_df.dropna()

roll_df.to_csv('data/rolls.zip', index=False)

print(len(roll_df['character'].unique()))
print(len(roll_df['type'].unique()))
print(len(roll_df['session'].unique()))

print(roll_df['character'].unique())

6
5
13
['Leslie' 'Viktor' 'Feyrn' 'Damaia' 'NPC' 'Kal']


In [117]:
roll_df = pd.read_csv('data/rolls.zip', parse_dates=['time'])

print(roll_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 798 entries, 0 to 797
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   time       798 non-null    datetime64[ns]
 1   character  798 non-null    object        
 2   session    798 non-null    int64         
 3   type       798 non-null    object        
 4   value      798 non-null    int64         
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 31.3+ KB
None


In [118]:
import plotly.express as px

last_session_df = roll_df[roll_df['time'] > datetime(2021,12,28)]
print(len(last_session_df))

kal_df =  last_session_df[last_session_df['character'] == 'Kal']
print(kal_df)

fig = px.histogram(last_session_df, x="value")
fig.show()

99
                   time character  session           type  value
699 2021-12-29 01:10:00       Kal       10   Spell/Attack      6
700 2021-12-29 01:10:00       Kal       11   Spell/Attack     14
701 2021-12-29 01:10:00       Kal       11   Spell/Attack      1
702 2021-12-29 01:10:00       Kal       11   Spell/Attack     14
703 2021-12-29 21:06:00       Kal       11  Ability Check     10
705 2021-12-29 21:26:00       Kal       11  Ability Check     20
706 2021-12-29 21:26:00       Kal       11  Ability Check      3
709 2021-12-29 21:53:00       Kal       11  Ability Check     15
711 2021-12-29 21:54:00       Kal       11  Ability Check      3
712 2021-12-29 21:54:00       Kal       11  Ability Check      2
716 2021-12-29 22:21:00       Kal       11  Ability Check      3
717 2021-12-29 22:21:00       Kal       11  Ability Check      1
719 2021-12-29 22:51:00       Kal       11  Ability Check      2
726 2021-12-30 00:47:00       Kal       11        Generic     22
727 2021-12-30 00:47:0

In [119]:
counts = roll_df.groupby(['character']).count()['time'].sort_values()
pd.set_option("display.max_rows", None, "display.max_columns", None)
print(counts)

character
NPC        12
Viktor     30
Damaia     65
Leslie    134
Kal       232
Feyrn     325
Name: time, dtype: int64


In [120]:
counts = roll_df.groupby(['type']).count()['time'].sort_values()
print(counts)

type
Initiative        21
Spell/Attack      27
Saving Throw      33
Ability Check    169
Generic          548
Name: time, dtype: int64


In [96]:
data = roll_df.groupby(['session','type']).count()['value'].reset_index(level=[0,1])
print(data)

    session           type  value
0         0        Generic    124
1         1        Generic    125
2         2        Generic     25
3         3        Generic     85
4         3   Spell/Attack      1
5         4  Ability Check     27
6         4        Generic      4
7         4     Initiative      1
8         4   Saving Throw      6
9         4   Spell/Attack      4
10        5  Ability Check     15
11        5        Generic     36
12        5     Initiative      2
13        5   Saving Throw      4
14        5   Spell/Attack      3
15        6  Ability Check     28
16        6        Generic     41
17        6     Initiative      2
18        6   Saving Throw      5
19        6   Spell/Attack      2
20        7  Ability Check     15
21        7        Generic      7
22        7     Initiative      2
23        7   Saving Throw      3
24        7   Spell/Attack      1
25        8  Ability Check     15
26        8        Generic      3
27        8     Initiative      9
28        8   

In [121]:
print(sum(roll_df['character'] == 'Kal'))

232
